# Introducción

Este *notebook* incluye:
- Pequeños ejemplos de uso de *pandas*.
- Un *DataSet* (de *PyTorch*) que almacena información de los archivos de audio con los cantos de las aves. Este *DataSet*, al solicitársele el i-ésimo *item*, devuelve un cacho del i-ésimo audio, un cacho de un j-ésimo audio, y un 0 o 1 si `i != j` o `i == j` respectivamente.
- Un *DataLoader* (de *PyTorch*) que envuelve al *DataSet* previamente descrito.
- 

### Importaciones

Importación de las bibliotecas a utilizar, y una pequeña descripción de cada una.

In [1]:
# pandas is an open source data analysis and manipulation tool.
import pandas as pd

# NumPy is for scientific computing with Python
import numpy as np

# PyTorch is an open source machine learning framework.
import torch

# PyTorch provides the torch.nn module to help us
# in creating and training of the neural network.
import torch.nn as nn

# PyTorch has two primitives to work with data:
# torch.utils.data.Dataset stores the samples and their corresponding labels.
# torch.utils.data.DataLoader wraps an iterable around the Dataset.
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# "The easiest way to use deep metric learning in your application".
# Written in PyTorch.
# https://github.com/KevinMusgrave/pytorch-metric-learning
from pytorch_metric_learning import losses

# librosa is for music and audio analysis; it provides
# the building blocks necessary to create music
# information retrieval systems.
import librosa

# Displays a spectrogram/chromagram/cqt/etc.
from librosa.display import specshow

# DEBUG
# 
from timeit import default_timer as timer

# Manejo de pseudo-aleatoriedad.
import random

# Manejo de funciones matemáticas.
import math

### Variables generales

Variables generales/globales que se utilizarán a lo largo del *notebook*. Conviene tener este apartado para consultarlas y modificarlas fácilmente.

In [2]:
# Cadena con la ubicación del archivo CSV que contiene
# el DataFrame con datos de los audios de aves.
birds_csv = "/media/birds/BirdsDataFrame.csv"

# Cadena con la ubicación de los archivos WAV y XML
# correspondientes a los audios de aves a procesar.
birds_path = "/media/birds/data/"

# Nombre de la columna, dentro del DataFrame,
# que contiene el nombre de los archivos de audio.
file_col_name = "FileName"

# DataFrame (de 'pandas') del archivo CSV dado.
birds_df = pd.read_csv(birds_csv)

# Los audios de aves se cortarán en cachos cuya longitud
# varíe entre len_min segundos y len_max segundos.
len_min = 1
len_max = 1

# Ancho y alto de cada espectrograma.
# TO-DO: ¿Es posible calcular esto mediante una fórmula? Resulta del size()/shape de aplicar "stft" al audio "y".
ancho,alto = 1025,87

# Frecuencia de muestreo a la cual TODOS los audios se
# muestrearán. Esto es necesario para que los vectores
# que representan a los audios tengan los mismos tamaños.
sr = 44100

# Probabilidad de que dos audios de aves (o
# cachos de audios) compartan cierta propiedad.
p_prop = 0.5

# Uso del GPU, si está disponible.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Utilizando {device} para el procesamiento de datos.")

# Variables asociadas a la Red Neuronal.
batch_size = 64 # Número de muestras que se tomarán por lote/epoch.
epochs = 10 # Veces que se recorrerá el DataSet entero.
lr = 0.01 # Learning Rate.
#momentum = 0.5 # The SGD momentum (default: 0.5) is the moving average of our gradients (helps to keep direction).

# Función a utilizar para procesar los audios de aves.
def librosa_process(path, cut, cut_len=None):
    """
    Función que carga un audio con Librosa y devuelve el vector
    unidimensional que representa al audio, y su frecuencia de muestreo.
    :param str path: Ruta donde se ubica el audio.
    :param bool cut: ¿Se cortará (y devolverá) sólo un cacho aleatorio del audio?
    :param float cut_len: Longitud del cacho de audio (si cut==True).
    """
    
    # Longitud del audio completo en segundos.
    audio_len = librosa.get_duration(filename=path)
    
    # Si se desea el audio completo, 'librosa' lo
    # cargará desde el inicio hasta el final.
    start = 0
    duracion = audio_len
        
    # Si se desea sólo un cacho del audio...
    if cut:
        
        # Determinamos la longitud del cacho
        # aleatorio de audio en segundos.
        duracion = cut_len if cut_len != None else random.uniform(len_min, len_max) # Rango [a,b].
        
        # Aseguramos que el audio completo es más
        # grande que el tamaño del cacho que queremos.
        assert audio_len > duracion
        
        # Definimos en dónde empezará
        # (aleatoriamente) el cacho de audio.
        start = random.uniform(0, audio_len-duracion) # Rango [a,b].
    
    # Obtenemos el audio-vector y su (nueva) frecuencia de muestreo.
    y, sampling_rate = librosa.load(path, sr=sr, offset=start, duration=duracion, mono=False)
    
    # Algunos audios fueron grabados en dos canales (stereo), y otros en
    # uno (mono). Convertimos los que fueron grabados en un canal en
    # audios de dos canales (al duplicar el único canal que tienen).
    if y.ndim == 1:
        y = np.repeat(y[np.newaxis, :], 2, axis=0)
    
    # Función no definida para audios que tienen más de dos canales.
    # Igual se lanza un error si los vectores no tienen la longitud adecuada (sr).
    assert(y.shape == (2, sr))
    
    # Short-time Fourier transform (STFT).
    # The STFT represents a signal in the time-frequency domain by computing
    # discrete Fourier transforms (DFT) over short overlapping windows.
    stft = librosa.stft(y)
    
    # This function (stft) returns a complex-valued matrix D such that
    # np.abs(D[..., f, t]) is the magnitude of frequency bin f at frame t.
    magnitude = np.abs(stft)
    
    # Converts an amplitude spectrogram to dB-scaled spectrogram.
    spectogram = librosa.amplitude_to_db(magnitude)
    
    # Devolvemos el espectrograma y su frecuencia de muestreo.
    return spectogram, sampling_rate

# Comprobaciones sobre las variables aquí definidas.
assert len_min <= len_max # Lógicamente, min<=max.
assert p_prop >= 0 and p_prop <= 1 # Las probabilidades se encuentran en este rango.

Utilizando cpu para el procesamiento de datos.


/home/alamina/.local/lib/python3.6/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


### _pandas_

Dado que el archivo `birds_csv` cuenta con *N* columnas `columna0,columna1,...,columnaN-1`, imprimimos a continuación el nombre de cada columna, enumerándolas desde cero.

**NOTA**: La primera columna no tiene nombre, por lo que *pandas*, al convertir el archivo CSV en un *DataFrame* mediante la función `read_csv()`, le asigna el nombre `Unnamed: 0`. Esta columna sirve para indexar a las entradas dentro del archivo CSV (no confundir con la columna 'index' cuyo propósito es indexar a los archivos de audio de otra manera).

In [3]:
# Por cada columna del DataFrame, imprimimos dicha columna.
for i,col in enumerate(birds_df.columns):
    print(f"{i}:{col}")

0:Unnamed: 0
1:index
2:Quality
3:ID
4:ClassID
5:Date
6:Time
7:Country
8:Locality
9:Latitude
10:Longitude
11:Elevation
12:Author
13:AuthorID
14:Content
15:Year
16:Family
17:Genus
18:Species
19:VernacularNames
20:FileName
21:BackgroundSpecies
22:CoincidenciaWavAlmacenados
23:sampling rate (kHz)
24:No.Muestras
25:Duración (s)


Ejemplificamos con la primera entrada del archivo al imprimir qué valor tiene asociado a cada columna.

In [4]:
# "iloc" permite indexar por posiciones mediante el uso de enteros.
# Por cada columna y valor en la primera línea, imprimimos ambos.
for col, val in birds_df.iloc[0].iteritems():
    print(f"{col}:{val}")

Unnamed: 0:0
index:1
Quality:1
ID:34204
ClassID:yyyyep
Date:2002-07-31
Time:10:30
Country:Venezuela
Locality:El Pauji road, Bolivar
Latitude:4.6
Longitude:-61.5
Elevation:750 - 950
Author:Peter Boesman
AuthorID:OOECIWCSWV
Content:song
Year:BirdCLEF2017
Family:Thamnophilidae
Genus:Terenura
Species:spodioptila
VernacularNames:Ash-winged Antwren
FileName:LIFECLEF2017_BIRD_XC_WAV_RN34204.wav
BackgroundSpecies:0
CoincidenciaWavAlmacenados:1
sampling rate (kHz):44.1
No.Muestras:825984
Duración (s):18.73


Hay algunas columnas tal que todas las entradas del archivo comparten un mismo valor dentro de esa columna. A continuación imprimimos los nombres de las columnas que cumplen ésto, así como el valor que todas las entradas comparten en dicha columna.

In [5]:
# Por cada columna del DataFrame...
for col in birds_df.columns:
    
    # Si todas las entradas tienen el mismo valor en dicha
    # columna, imprimimos la columna y el valor correspondiente.
    if (birds_df[col] == birds_df[col][0]).all():
        pass # DEBUG: Borrar esta línea y descomentar la siguiente.
        #print(f"{col}:{birds_df[col][0]}")

# Definición del *DataSet*

Creamos el *DataSet* de *PyTorch* que guarda y maneja los datos de los archivos de audio (que contienen los cantos de las aves).

In [6]:
class CustomBirdDataset(Dataset):
    """
    Dataset de audios de aves.
    """
    
    def __init__(self, df, process_func, audio_path, transform=None, target_transform=None):
        """
        The __init__ function is run once when instantiating the Dataset object.
        """
        
        # 'df' es el DataFrame a almacenar.
        self.df = df
        
        # 'process_func' toma la ruta de un audio a procesar, y lo procesa.
        self.process_func = process_func
        
        # 'audio_path' es la ruta donde se ubican los archivos de audio.
        self.audio_path = audio_path
        
        # 'transform' and 'target_transform' modify the samples and labels respectively.
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        """
        The __len__ function returns the number of samples in our dataset.
        """
        return len(self.df)

    def __getitem__(self, idx=None):
        """
        The __getitem__ function loads and returns a sample from the dataset at the given index 'idx'.
        """
        
        # Si no se especifica un índice, se toma una muestra aleatoria.
        if idx == None:
            idx = random.randrange(0, birds_ds.__len__()) # Rango [a,b).
        
        # Obtenemos la idx-ésima línea del DataFrame almacenado.
        # Y el nombre del archivo de audio a procesar.
        item = self.df.iloc[idx]
        filename = item[file_col_name]
        
        # Procesamos el primer cacho de audio.
        x,_ = self.process_func(self.audio_path+filename, True)
        
        # Si se desea que ambos cachos de audio compartan la propiedad,
        # sólo dejamos la etiqueta como "1", y volvemos a procesar
        # el mismo archivo de audio de manera aleatoria (más adelante).
        if (random.random() < p_prop):
            target = 1
            
        # Si, por otro lado, se desea que los cachos no compartan la
        # propiedad, dejamos la etiqueta como "0", y buscamos otro
        # archivo de audio para procesar.
        else:
            target = 0
            
            # Guardamos la especie del ave del primer cacho de audio.
            primera_especie = item["Species"]
            
            # Quitamos el primer archivo de audio (que ya fue procesado) del
            # DataFrame (temporalmente), obtenemos algún renglón aleatorio de
            # este nuevo DataFrame (sample() devuelve un DataFrame, por lo que
            # es necesario tomar el primer renglón con iloc[0]), y obtenemos
            # el nombre del nuevo archivo de audio a procesar.
            item = self.df.drop(idx).sample().iloc[0]
            filename = item[file_col_name]
                  
            # Guardamos la especie del ave del segundo cacho de audio.
            segunda_especie = item["Species"]
            
            # TO-DO: Checar especie.
            # Si son la misma especie,
            # ¿sigo buscando otro segundo cacho de audio,
            # o cambio el "target" a 1?
            #print(f"El primer cacho de audio pertenece a un ave {primera_especie}, y el segundo pertenece a un ave {segunda_especie}.")
            if primera_especie == segunda_especie:
                target = 1
                print(f"DEBUG: Analizar este caso.")

        # Procesamos el segundo cacho de audio.
        y,_ = self.process_func(self.audio_path+filename, True)
            
        # NOTA:
        # Aún no se define el uso para 'transform' y 'target_transform'.
        # Una propuesta es que 'transform' sustituya a 'process_func'.
        
        # Devolvemos el primer cacho de audio, el segundo cacho de audio,
        # y la etiqueta que indica si ambos comparten (1) o no (0) la propiedad.
        return x, y, target

### Ejemplo del *DataSet*

Cargamos el *DataSet* al pasarle:
- El *DataFrame* creado previamente con *pandas*.
- La función a utilizar para procesar los audios.
- La ruta del directorio en el cual se encuentran los archivos de audio.

Y obtenemos una muestra aleatoria del *DataSet* mediante su función `__getitem__()`.

In [7]:
birds_ds = CustomBirdDataset(birds_df, librosa_process, birds_path)
birds_ds.__getitem__()

(array([[[  8.692846 ,  -5.360284 ,   5.0297284, ..., -55.431656 ,
          -55.431656 , -55.431656 ],
         [ 12.752359 ,  15.000231 ,  11.44207  , ..., -55.431656 ,
          -55.431656 , -55.431656 ],
         [ 16.47116  ,  20.661308 ,  17.366951 , ..., -55.431656 ,
          -55.431656 , -55.431656 ],
         ...,
         [-49.317207 , -55.41533  , -55.431656 , ..., -55.431656 ,
          -55.431656 , -55.431656 ],
         [-49.31803  , -55.431656 , -55.431656 , ..., -55.431656 ,
          -55.431656 , -55.431656 ],
         [-49.31692  , -55.408073 , -55.431656 , ..., -55.431656 ,
          -55.431656 , -55.431656 ]],
 
        [[  0.6841111,   3.4911704,  -4.4718924, ..., -55.431656 ,
          -55.431656 , -55.431656 ],
         [  5.124631 ,  12.259501 ,  14.1137085, ..., -55.431656 ,
          -55.431656 , -55.431656 ],
         [ 10.233149 ,  14.724383 ,  17.364658 , ..., -55.431656 ,
          -55.431656 , -55.431656 ],
         ...,
         [-53.97046  , -55.431656

# Definición del *DataLoader*

Creamos el *DataLoader* de *PyTorch* que envuelve el *DataSet* previamente definido.

In [8]:
# TO-DO: Verificar el argumento 'shuffle'.
birds_dl = DataLoader(birds_ds, batch_size=batch_size, shuffle=True)

### Ejemplo del **DataLoader**

El *DataLoader* contiene listas (que regresa la función `__getitem__()` correspondiente al *DataSet*). Estas listas contienen los lotes de tamaño `batch_size` y, para abarcar todos los datos, contiene aproximadamente `tamaño_de_todos_los_datos/batch_size` listas.

In [9]:
print(f"- Tamaño del DataSet (de PyTorch) = {len(birds_ds)} = {len(birds_df)} = Tamaño del DataFrame (de pandas)")
print(f"- Tamaño del DataLoader (de PyTorch): {len(birds_dl)}")
iterador = iter(birds_dl)
primer_lote = next(iterador)
print(f"- Tamaño de la primera lista del DataLoader: {len(primer_lote)}")
print(f"- Tamaño de los elementos de la primera lista: {len(primer_lote[0])} {len(primer_lote[1])} {len(primer_lote[2])}")
print(f"- Tamaño del DataLoader por el tamaño de cada lote: {len(birds_dl)*batch_size} ≈ {len(birds_ds)}")

- Tamaño del DataSet (de PyTorch) = 3277 = 3277 = Tamaño del DataFrame (de pandas)
- Tamaño del DataLoader (de PyTorch): 52
- Tamaño de la primera lista del DataLoader: 3
- Tamaño de los elementos de la primera lista: 64 64 64
- Tamaño del DataLoader por el tamaño de cada lote: 3328 ≈ 3277


Imprimimos datos sobre el primer lote para ejemplificar.

In [10]:
primeros_cachos, segundos_cachos, labels = primer_lote
print(f"- Los primeros cachos de audio del primer lote tienen tamaño: {primeros_cachos.size()}")
print(f"- Los segundos cachos de audio del primer lote tienen tamaño: {segundos_cachos.size()}")
print(f"- Las etiquetas del primer lote tienen tamaño: {labels.size()}")
print(f"- Etiquetas del primer lote: {labels}")
print(f"- Primeros cachos del primer lote: {primeros_cachos}")
print(f"- Segundos cachos del primer lote: {segundos_cachos}")

- Los primeros cachos de audio del primer lote tienen tamaño: torch.Size([64, 2, 1025, 87])
- Los segundos cachos de audio del primer lote tienen tamaño: torch.Size([64, 2, 1025, 87])
- Las etiquetas del primer lote tienen tamaño: torch.Size([64])
- Etiquetas del primer lote: tensor([1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1,
        1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1])
- Primeros cachos del primer lote: tensor([[[[-14.8117, -10.2171, -19.3774,  ..., -14.4443, -10.9739,  -6.6863],
          [ -3.7802,  -1.5464,  -4.3461,  ...,  -3.4473,  -6.8875,  -2.4399],
          [ -0.3803,   2.7507,   0.3256,  ...,   2.4394,   3.2212,   0.9621],
          ...,
          [-40.1824, -40.1824, -40.1824,  ..., -40.1824, -40.1824, -33.2413],
          [-40.1824, -40.1824, -40.1824,  ..., -40.1824, -40.1824, -33.3233],
          [-40.1824, -40.1824, -40.1824,  ..., -40.1824, -40.1824,

# Definición de la Red Neuronal

In [11]:
# nn.Module is the base class for all neural network modules.
# Our models should also subclass this class.
# Modules can also contain other Modules, allowing to nest them in a tree structure.
class RN(nn.Module):
    """
    TO-DO: Terminar esta documentación.
    """
    
    #This defines the structure of the NN.
    def __init__(self):
        """
        Inicialización de la Red Neuronal.
        Aquí se define su estructura.
        """
        
        #
        super().__init__()
        
        # Inicio de las capas convolucionales.
        conv_layers = []
        
        # Primera capa convolucional.
        self.conv1 = nn.Conv2d(2, batch_size, kernel_size=(10,10))
        self.relu1 = nn.ReLU()
        self.mp1 = nn.MaxPool2d(2)
        conv_layers += [self.conv1, self.relu1, self.mp1]
        
        # DEBUG
        # x after mp1: torch.Size([64, 64, 469, 39])
        # "weight of size [out, in, kHeight, kWidth], expected input[64, 64, 469, 39] to have 'out' channels"
        
        # Segunda capa convolucional.
        self.conv2 = nn.Conv2d(batch_size, batch_size, kernel_size=(7,7))
        self.relu2 = nn.ReLU()
        self.mp2 = nn.MaxPool2d(2)
        conv_layers += [self.conv2, self.relu2, self.mp2]
        
        # DEBUG
        # x after mp2: torch.Size([64, 64, 191, 16])
        
        # Tercera capa convolucional.
        self.conv3 = nn.Conv2d(batch_size, batch_size, kernel_size=(2,2))
        self.relu3 = nn.ReLU()
        self.avg_pool3 = nn.AvgPool2d(kernel_size=(10,10))
        conv_layers += [self.conv3, self.relu3, self.avg_pool3]
        
        # DEBUG
        # x after relu3: torch.Size([64, 64, 105, 15])
        
        # Fin de las capas convoluciones.
        self.conv = nn.Sequential(*conv_layers)
        
        # Inicio de las capas lineales (fully-connected).
        fc_layers = []
        
        # Primera capa lineal.
        self.fc1 = nn.Linear(1600,1)
        fc_layers += [self.fc1]
        
        # Fin de las capas lineales.
        self.fc = nn.Sequential(*fc_layers)
    
    def invididual_process(self, z):
        
        start = timer()
        z = self.conv(z)
        end = timer()
        print(f"\t\t[time] Capas convolucionales: {end-start}s") # DEBUG
        
        start = timer()
        z = z.view(z.shape[0], -1)
        end = timer()
        print(f"\t\t[time] Aplanando: {end-start}s") # DEBUG
        
        start = timer()
        z = self.fc(z)
        end = timer()
        print(f"\t\t[time] Capas lineales: {end-start}s") # DEBUG
        
        return z

    def forward(self, x, y):
        
        print("\tProcesando x...") # DEBUG
        x = self.invididual_process(x)
        print("\tProcesando y...") # DEBUG
        y = self.invididual_process(y)
        
        diff = torch.abs(x - y)
        
        start = timer()
        activate = nn.Sigmoid()
        end = timer()
        print(f"\t[time] Activating: {end-start}s") # DEBUG
        
        diff = activate(diff)
        
        return diff

# Entrenamiento de la Red Neuronal

In [12]:
###### Definimos el modelo.
red = RN().to(device)

# TO-DO: Describir 'optimizer'.
# TO-DO: Escoger 'optimizer' adecuado.
#optimizer = torch.optim.SGD(red.parameters(), lr=lr, momentum=momentum) # Usado por MNIST Colab.
optimizer = torch.optim.Adam(red.parameters(), lr = lr) # Usado por AlexNet (TMLoss y MSELoss).

# TO-DO: Describir 'loss'.
# TO-DO: Escoger 'loss' adecuado.
loss_func = losses.ContrastiveLoss().to(device)

# red.train() le indica al modelo que está siendo entrenado.
# Esto ayuda con capas como Dropout y BatchNorm, que están
# diseñadas para comporsarse distinto durante entrenamiento
# y evaluación.
red.train()

def train(epoch):
    """
    TO-DO: Terminar esta documentación.
    """
    
    # Imprimimos el número de epoch.
    print(f"Epoch {epoch}...")
    for batch_idx, data in enumerate(birds_dl):
        
        # DEBUG.
        print(f"\nProcesando lote {batch_idx+1}/{len(birds_dl)}...")
        
        # 'data' es una lista que representa un lote:
        # data[0] contiene los primeros cachos de audio.
        # data[1] contiene los segundos cachos de audio.
        # data[2] contiene las etiquetas.
        for i,d in enumerate(data):
            data[i] = d.to(device)
            
        # Vaciamos los gradientes para este lote.
        optimizer.zero_grad()
        # In PyTorch, for every mini-batch during the training phase,
        # we typically want to explicitly set the gradients to zero
        # before starting to do backpropragation (i.e., updating the
        # Weights and biases) because PyTorch accumulates the gradients
        # on subsequent backward passes.
        
        # Metemos los datos a la red neuronal.
        output = red(data[0], data[1])
        
        # TO-DO: Describir.
        
        start = timer()
        loss = loss_func(output, data[2])
        end = timer()
        print(f"[time] Loss function: {end-start}s") # DEBUG
        
        start = timer()
        loss.backward() #dloss/dx for every variable
        end = timer()
        print(f"[time] Loss backward: {end-start}s") # DEBUG
        
        optimizer.step() #to do a one-step update on our parameter.
        
        
        # DEBUG:
        print(f"\tLoss: {loss}")
        print(f"\tEtiquetas: {data[2]}")
        print(f"\tOutput: {output.size()} - {output}")

# Ejecutamos el entrenamiento definido, "epochs" veces.
for epoch in range(1, epochs+1): # Rango [a, b)
    train(epoch)

Epoch 1...

Procesando lote 1/52...
	Procesando x...
		[time] Capas convolucionales: 5.627240557223558s
		[time] Aplanando: 0.00024266913533210754s
		[time] Capas lineales: 0.00024785101413726807s
	Procesando y...
		[time] Capas convolucionales: 5.620529379695654s
		[time] Aplanando: 9.672343730926514e-05s
		[time] Capas lineales: 0.0001453608274459839s
	[time] Activating: 5.167350172996521e-05s
[time] Loss function: 0.001672394573688507s
[time] Loss backward: 24.153069250285625s
	Loss: 1.0
	Etiquetas: tensor([0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
        0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
        1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1])
	Output: torch.Size([64, 1]) - tensor([[0.5976],
        [0.5772],
        [0.5752],
        [0.8260],
        [0.7410],
        [0.7050],
        [0.5861],
        [0.5427],
        [0.5060],
        [0.6248],
        [0.5450],
        [0.5004],
        [0.5241],
        [0

# DEBUG

Esta celda y las siguientes son para testear. Han de ser eliminadas cuando se limpie el código de este *notebook*.

In [13]:
f = birds_df.iloc[5][file_col_name]
librosa_process(birds_path+f, True)

(array([[[-30.787476, -35.89453 , -38.152206, ..., -38.152206,
          -38.152206, -38.152206],
         [-30.680197, -36.19075 , -38.152206, ..., -38.152206,
          -38.152206, -38.152206],
         [-30.480244, -35.340866, -38.152206, ..., -38.152206,
          -38.152206, -38.152206],
         ...,
         [-38.152206, -38.152206, -38.152206, ..., -38.152206,
          -38.152206, -38.152206],
         [-38.152206, -38.152206, -38.152206, ..., -38.152206,
          -38.152206, -38.152206],
         [-38.152206, -38.152206, -38.152206, ..., -38.152206,
          -38.152206, -38.152206]],
 
        [[-30.93338 , -36.189285, -38.152206, ..., -38.152206,
          -38.152206, -37.612118],
         [-30.77877 , -36.308826, -38.152206, ..., -38.152206,
          -38.152206, -37.46538 ],
         [-30.554302, -35.28008 , -38.152206, ..., -38.152206,
          -38.152206, -37.29844 ],
         ...,
         [-38.152206, -38.152206, -38.152206, ..., -38.152206,
          -38.152206, -3

In [14]:
random.uniform(1,1)

1.0

In [15]:
print(*range(0,10))
print(*range(1,10))

0 1 2 3 4 5 6 7 8 9
1 2 3 4 5 6 7 8 9
